<a href="https://colab.research.google.com/github/singhsidhukuldeep/training-BERT-from-scratch/blob/main/BERT_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Reference: https://huggingface.co/blog/how-to-train

In [ ]:
# https://huggingface.co/nlp/viewer/?dataset=wiki_split

# https://huggingface.co/datasets/wiki_split

# 1 Million lines/ 170 MB

!wget -c https://github.com/google-research-datasets/wiki-split/raw/master/train.tsv.zip

--2020-11-02 11:09:21--  https://github.com/google-research-datasets/wiki-split/raw/master/train.tsv.zip
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/google-research-datasets/wiki-split/master/train.tsv.zip [following]
--2020-11-02 11:09:21--  https://raw.githubusercontent.com/google-research-datasets/wiki-split/master/train.tsv.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 96439435 (92M) [application/zip]
Saving to: ‘train.tsv.zip’

train.tsv.zip       100%[===================>]  91.97M   143MB/s    in 0.6s    

2020-11-02 11:09:23 (143 MB/s) - ‘train.tsv.zip’ saved [96439435/96439435]


In [ ]:
!ls
!unzip train.tsv.zip

sample_data  train.tsv.zip
Archive:  train.tsv.zip
  inflating: train.tsv               


In [ ]:
!head -10 train.tsv

' '' As she translates from one language to another , she tries to find the appropriate wording and context in English that would correspond to the work in Spanish her poems and stories started to have differing meanings in their respective languages .	' '' As she translates from one language to another , she tries to find the appropriate wording and context in English that would correspond to the work in Spanish . <::::> Ergo , her poems and stories started to have differing meanings in their respective languages .
' '' BDSM is solely based on consensual activities , and based on its system and laws , the concepts presented by de Sade are not agreed upon the BDSM culture , even though they are sadistic in nature .	' '' BDSM is solely based on consensual activities , and based on its system and laws . <::::> The concepts presented by de Sade are not in accordance with the BDSM culture , even though they are sadistic in nature .
' '' Critics criticized the use of the disputed figures by

In [ ]:
import pandas as pd

df = pd.read_csv('train.tsv', sep='\t', header=None)

In [ ]:
df.head()

,0,1
0,' '' As she translates from one language to an...,' '' As she translates from one language to an...
1,' '' BDSM is solely based on consensual activi...,' '' BDSM is solely based on consensual activi...
2,' '' Critics criticized the use of the dispute...,' '' Critics criticized the use of the dispute...
3,' '' Do Re Mi '' ' is a song by Kurt Cobain of...,"' '' Do Re Mi '' ' is a song by Kurt Cobain , ..."
4,"' '' For Robert Price '' docetism '' , togethe...","' '' For Robert Price '' docetism '' , togethe..."


In [ ]:
clr = lambda x: x.replace("' '' ", '')
df[0] = df[0].apply(clr)
df.drop([1], axis=1, inplace=True)
df.head()

,0
0,As she translates from one language to another...
1,BDSM is solely based on consensual activities ...
2,Critics criticized the use of the disputed fig...
3,Do Re Mi '' ' is a song by Kurt Cobain of the ...
4,"For Robert Price '' docetism '' , together wit..."


In [ ]:
df[0].to_csv('train.txt', index=False)

In [ ]:
# We won't need TensorFlow here
# !pip uninstall -y tensorflow
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'

Uninstalling tensorflow-2.3.0:
  Successfully uninstalled tensorflow-2.3.0
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-iwcm20s2
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-iwcm20s2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 890kB 4.4MB/s 
     |████████████████████████████████| 2.9MB 13.8MB/s 
     |████████████████████████████████| 1.1MB 40.0MB/s 
  Created wheel for transformers: filename=transformers-3.4.0-cp36-none-any.whl size=1285611 sha256=71918956a1daa76f516c2781f3ec14812204bd30adb93d4ca5b9cfae591ad77b
  Stored in directory: /tmp/pip-ephem-wheel-cache-9er4jrfi/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
Successfully built transformers
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=db809b6f398987e0d7d91846e

In [ ]:
%%time 
from pathlib import Path

from tokenizers import ByteLevelBPETokenizer

paths = [str(x) for x in Path(".").glob("**/*.txt")]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=20_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

CPU times: user 3min 54s, sys: 991 ms, total: 3min 55s
Wall time: 2min 1s


In [ ]:
!mkdir BERTboi
tokenizer.save_model("BERTboi")

['BERTboi/vocab.json', 'BERTboi/merges.txt']

In [ ]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "./BERTboi/vocab.json",
    "./BERTboi/merges.txt",
)

In [ ]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=32)

In [ ]:
tokenizer.encode("This is America")

Encoding(num_tokens=5, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [ ]:
tokenizer.encode("This is America").tokens

['<s>', 'This', 'Ġis', 'ĠAmerica', '</s>']

In [ ]:
!nvidia-smi

Mon Nov  2 11:37:31 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=20_000,
    max_position_embeddings=400,
    num_attention_heads=2,
    num_hidden_layers=3,
    type_vocab_size=1,
)

In [ ]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./BERTboi", max_len=32)

In [ ]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [ ]:
model.num_parameters()

37545248

In [ ]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./train.txt",
    block_size=128,
)

CPU times: user 3min 3s, sys: 4.37 s, total: 3min 7s
Wall time: 1min 49s


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./BERTboi",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=sc,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

/usr/local/lib/python3.6/dist-packages/transformers/trainer.py:281: FutureWarning: Passing `prediction_loss_only` as a keyword argument is deprecated and won't be possible in a future version. Use `args.prediction_loss_only` instead. Setting `args.prediction_loss_only=True
  FutureWarning,


In [ ]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss
500,7.355470
1000,6.859652
1500,6.705604
2000,6.595646
2500,6.494686
3000,6.357398
3500,6.208945
4000,6.054441
4500,5.898809
5000,5.761188


CPU times: user 51min 14s, sys: 39min 16s, total: 1h 30min 30s
Wall time: 1h 31min 1s


TrainOutput(global_step=15468, training_loss=5.5329965493276445)

In [ ]:
trainer.save_model("./BERTboi")

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./BERTboi",
    tokenizer="./BERTboi"
)

Some weights of RobertaModel were not initialized from the model checkpoint at ./BERTboi and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
fill_mask("Apples are good for <mask>")

[{'score': 0.012861344031989574,
  'sequence': '<s>Apples are good for example</s>',
  'token': 2438,
  'token_str': 'Ġexample'},
 {'score': 0.008665323257446289,
  'sequence': '<s>Apples are good for them</s>',
  'token': 821,
  'token_str': 'Ġthem'},
 {'score': 0.0066885086707770824,
  'sequence': '<s>Apples are good for people</s>',
  'token': 1136,
  'token_str': 'Ġpeople'},
 {'score': 0.006021216511726379,
  'sequence': '<s>Apples are good for different</s>',
  'token': 1615,
  'token_str': 'Ġdifferent'},
 {'score': 0.005974438972771168,
  'sequence': '<s>Apples are good for."</s>',
  'token': 293,
  'token_str': 'Ġ."'}]